<a href="https://colab.research.google.com/github/snehith26/natural-language-processing/blob/master/Text_Generation_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>